# Linear Programming

This notebook:

- an introduction to linear programming,
- food/diet example (show),
- example of dispatching an energy system with scenario analysis (show - should be do together!),
- port/market exercise (do alone).

Make sure to at least [glance through the notes](https://github.com/ADGEfficiency/teaching-monolith/blob/master/linear-programming/README.md) before working through these examples.


## Linear programming with PuLP

We can do linear programming in Python using [PuLP](https://pythonhosted.org/PuLP/) - a library that abstracts away the mechanics of solving linear programs, and let's us focus on the three components that relate to our business problem:

1. objective function we want to make big or small,
2. variables we can change,
3. constraints we need to follow.

There are many other solvers - most Python LP libraries will wrap around a lower level implementation in a complied langugae (I would hope!).


## Example - Diet Problem

Adapted from [Linear programming - Michel Goemans](https://math.mit.edu/~goemans/18310S15/lpnotes310.pdf).

> In the diet model, a list of available foods is given together with the nutrient content and the cost
per unit weight of each food. A certain amount of each nutrient is required per day. 

> We live in a civilization with just two types of fruit (apples and oranges) and three types of nutrients (starch, proteins, vitamins).

> We want a diet that is cheap, while satasifying our dietary requirements.  

> We have a requirement per day of 8g of starch, 15g of proteins and 3g of vitamins.

| | Starch [kg/kg] | Proteins [kg/kg] | Vitamins [kg/kg] | Cost [$/g] |
|---|---|---|---|---|
|apples| 5 | 4 | 2| 0.6 |
|oranges| 7 | 2 | 1| 0.35 |


### Formulating as a linear problem

Let's map this problem onto the three components of a linear program:

1. objective function = minimize the cost of our diet,
2. variables = the amount of apples & oranges we choose to eat,
3. constraints = our daily requirements of starch, protien and vitamins.


## The PuLP api

Below is a full linear program for the problem above:

In [ ]:
!pip install pip -Uq
!pip install pulp numpy -Uq

from pulp import LpProblem, LpMinimize, LpVariable, LpStatus

#  we can do min or max in pulp - we need to say which 
prob = LpProblem('diet-cost-minimization', LpMinimize)

#  our variables - the things we can change
apples = LpVariable('apples', cat='Integer')
oranges = LpVariable('oranges', cat='Integer')

#  add the objective function - how expensive our diet is
prob += apples * 0.6 + oranges * 0.35

#  add our dietary constraints
#  starch
prob += apples * 5 + oranges * 7 >= 8
#  protein
prob += apples * 4 + oranges * 2 >= 15
#  vitamins
prob += apples * 2 + oranges * 1 >= 3

#  run problem & show results
prob.solve()
print(f'Problem is {LpStatus[prob.status]}, your diet cost is {prob.objective.value()}')
for v in (apples, oranges):
    print(f'{v.name}: {v.varValue}')

What has happened here?  Our program has done something quite wierd!

Let's try again, putting some more constraints on our variables.

You could also achieve this by an explicit constraint (something like `prob += apples >=0` for a minimum of zero).

`puLp` lets us do these constraints (that the the amount of fruit we eat should be `>=0`) when we create the variables:

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, LpStatus

prob = LpProblem('diet cost minimization', LpMinimize)
apples = LpVariable('apples', cat='Integer', lowBound=0, upBound=None)
oranges = LpVariable('oranges', cat='Integer', lowBound=0, upBound=None)

#  add the objective function
prob += apples * 0.6 + oranges * 0.35

#  add constraints
prob += apples * 5 + oranges * 7 >= 8
prob += apples * 4 + oranges * 2 >= 15
prob += apples * 2 + oranges * 1 >= 3

prob.solve()
print(f'Problem is {LpStatus[prob.status]}, your diet cost is {prob.objective.value()}')
for v in (apples, oranges):
    print(f'{v.name}: {v.varValue}')

Now we have a model - we can start to do some sensitvitiy analysis (do manually with class).

## Example - electricity system dispatch

Three assets in our grid:

- wind turbine,
- gas turbine,
- big bad coal.

In [ ]:
from dataclasses import dataclass

@dataclass
class Asset:
    name: str
    price: float
    carbon_intensity: float
    limit: int

assets = [
    #  name, price $/MWh, carbon intensity tC/MWh
    ('wind', 30, 0.05, 100),
    ('gas', 70, 0.1, 50),
    ('coal', 50, 0.1, 100),
]
assets = [Asset(*a) for a in assets]

problem = LpProblem('cost minimization', LpMinimize)

#  make one variable per asset
variables = [LpVariable(a.name, 0, a.limit) for a in assets]

Let's create an objective function based on price:

In [ ]:
problem += sum([a.price * v for a, v in zip(assets, variables)])

And place one constraint - that the sum of our generation is equal to demand:

In [ ]:
demand = 10
problem += sum(variables) == demand

problem.solve()
print(LpStatus[problem.status])

for v in variables:
    print('{} {}'.format(v.name, v.varValue))

Now we have a model - we can use it for scenario analysis:

In [ ]:
for demand in [10, 50, 100]:
    
    problem = LpProblem('cost minimization', LpMinimize)
    assets = [
        #  name, price $/MWh, carbon intensity tC/MWh
        ('wind', 30, 0.05, 25),
        ('gas', 70, 0.1, 50),
        ('coal', 50, 0.1, 100),
    ]
    assets = [Asset(*a) for a in assets]
    variables = [LpVariable(a.name, 0, a.limit) for a in assets]
    
    problem += sum([a.price * v for a, v in zip(assets, variables)])
    problem += sum(variables) == demand
    problem.solve()
    print(LpStatus[problem.status])

    for v in variables:
        print('{} {}'.format(v.name, v.varValue))
    print('')

We can also look at what changing prices does to our dispatch:

In [ ]:
demand = 50
for coal_price in [10, 50, 100]:
    
    problem = LpProblem('cost minimization', LpMinimize)
    assets = [
        #  name, price $/MWh, carbon intensity tC/MWh
        ('wind', 30, 0.05, 25),
        ('gas', 70, 0.1, 50),
        ('coal', coal_price, 0.1, 100),
    ]
    assets = [Asset(*a) for a in assets]
    variables = [LpVariable(a.name, 0, a.limit) for a in assets]
    
    problem += sum([a.price * v for a, v in zip(assets, variables)])
    problem += sum(variables) == demand
    problem.solve()
    print(LpStatus[problem.status])

    for v in variables:
        print('{} {}'.format(v.name, v.varValue))
    print('')

Now time to do a problem on your own!

## Transportation problem

$P$ ports
- each port has a capacity, measured in number of units

$M$ markets
- each market has a demand, measured in number of units

Each $(P,M)$ pair has a cost - the cost to transport goods from port $P$ to market $M$.

We want to find the lowest cost way to supply all our market demands from our ports.

We can map this problem directly onto the definition of linear programming:

- objective function = ?
- variables = ?
- constraints = ?

## Exercise

Build a linear program to solve the transportation problem:

In [ ]:
ports = [20, 30, 30, 50]
markets = [20, 10, 5]

#  one price for a port-market pair - just randomly make prices
np.random.seed(42)
pm_cost = np.random.uniform(0, 1, size=len(ports) * len(markets)).reshape(len(ports), len(markets))

We can access the cost to trade from a port to a market by indexing `pm_cost[port, market]`:

In [ ]:
#  trade from port 0 to market 0
pm_cost[0, 0]

In [ ]:
#  trade from port 3 to market 2
pm_cost[3, 2]

Feel free to look at the answer by commenting out the two lines below:

In [ ]:
#from answers import transportation
#transportation??